In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

from shapely.geometry import shape, Point, Polygon
from shapely.ops import unary_union

import rtree as rt

%matplotlib inline
pd.set_option("display.max.columns", None)

### PurpleAir Data descriptors
##### Determined
- pm = current PM2.5 reading
- pm1 = raw PM1 reading
- pm_10 = raw PM10 reading
- pm_0 = current PM2.5 reading
- pm_1 = 10 minute PM2.5 average
- pm_2 = 30 minute PM2.5 average
- pm_3 = 1 hour PM2.5 average
- pm_4 = 6 hour PM2.5 average
- pm_5 = 24 hour PM2.5 average
- pm_6 = One week PM2.5 average
- p1 = Particles >= 0.3 µm
- p2 = Particles >= 0.5 µm
- p3 = Particles >= 1.0 µm
- p4 = Particles >= 2.5 µm
- p5 = Particles >= 5.0 µm
- p6 = Particles >= 10.0 µm
- flags = Data flagged for unusually high readings
- age = Sensor data age (when data was last received) in minutes
- isOwner = Currently logged in user is the sensor owner
- Adc = The voltage reading on the analog input of the control board

##### Undetermined
- conf
- Icon
- Voc
- Ozone1
- CH

## Functions

In [3]:
def sensor_data():
    # Use 'experimental' data from purpleair
    url = 'https://www.purpleair.com/data.json'
    data = requests.get(url).json()

    sensors_df = pd.DataFrame(data['data'], columns=data['fields'])
    sensors_df = sensors_df.set_index('ID')

    # Drop sensors with data more than 12 hours old
    sensors_df = sensors_df[sensors_df['age'] <= 720.0]

    # Drop sensors with missing data
    sensors_df = sensors_df[~sensors_df['pm'].isnull()]

    # Drop indoor sensors
    sensors_df = sensors_df[sensors_df['Type'] == 0]

    # Drop sensors > 3 standard deviations from the mean
    sensors_df = sensors_df[sensors_df['pm'] < (2 * sensors_df['pm'].std())]

    # Uses current PM2.5 reading ('pm')
    cols_to_keep = ['pm', 'Lat', 'Lon']
    cols_to_drop = [col for col in sensors_df.columns if col not in cols_to_keep]

    sensors_df = sensors_df.drop(cols_to_drop, axis=1)
    return sensors_df

# Returns dataframe of active purpleair sensors contained within LA County
def los_angeles_county_sensors(sensors_df):
    # Set boolean to use LAC shape or LAC bounding box (includes some Orange County and Ventura County)
    use_shape = False
    
    # Initialize rtree index
    idx = rt.index.Index()
    
    # Get LA County geometries in JSON form
    url = 'https://opendata.arcgis.com/datasets/10f1e37c065347e693cf4e8ee753c09b_15.geojson'
    shapes = requests.get(url).json()
    
    la_county_indeces = []
    
    # Union of individual LA County shapes. Speeds up check if sensor is in LAC.
    polygons = [shape(feature['geometry']) for feature in shapes['features']]
    lac_shape = unary_union(polygons)
    lac_bounds = lac_shape.bounds
    
    for index, row in sensors_df.iterrows():
        if ~np.isnan(row.Lon) and ~np.isnan(row.Lat):
            
            if use_shape:
                # Build Point object from sensor's Lon and Lat (in that order!) values
                point = Point(row.Lon, row.Lat)

                # Populate rtree index with point bounds
                idx.insert(int(index), point.bounds, point)
            else: 
                if lac_bounds[0] <= row.Lon and row.Lon <= lac_bounds[2] and lac_bounds[1] <= row.Lat and row.Lat <= lac_bounds[3]:
                    la_county_indeces.append(int(index))
    
    if use_shape:
        # get list of fids where bounding boxes intersect
        fids = [i for i in idx.intersection(lac_shape.bounds, objects=True)]

        # access the features that those fids reference
        for fid in fids:
            # check the geometries intersect, not just their bboxs
            if lac_shape.contains(fid.object):
                la_county_indeces.append(fid.id)
                    
    return sensors_df.loc[la_county_indeces]

# Census Tract data is provided by LATimes (not ideal), but is complete.
def la_county_census_tracts():
    use_url = False
    
    if use_url:
        # Read from URL
        test_url = 'http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/census-tracts-2012.geojson'
        shape_data = requests.get(test_url).json()
    else:
        # Open saved file
        with open('census-tracts-2012.geojson') as f:
            shape_data = json.load(f)
        
    census_tract_df = pd.DataFrame()
    census_tract_df['tract'] = [int(feature['properties']['name']) for feature in shape_data['features']]
    census_tract_df['coordinates'] = [feature['geometry']['coordinates'] for feature in shape_data['features']]
    census_tract_df['shape'] = [shape(feature['geometry']) for feature in shape_data['features']]
    census_tract_df.set_index('tract', inplace=True)
    return census_tract_df

def find_nearest_sensors(la_county_sensors_df, census_tract_df, n=3):
    # Populate rtree index with sensor points
    idx = rt.index.Index()
    for index, row in la_county_sensors_df.iterrows():
        idx.insert(int(index), row['point'].bounds)
        
    # Use rtree index's nearest function to find indexed point closest to the bounding box of the census tract
    nearest_sensors = []
    for index, row in census_tract_df.iterrows():
        centroid = row['shape'].centroid

        nearest = list(idx.nearest(centroid.bounds, n))[:n]
        nearest_sensors.append(nearest)
    census_tract_df['nearest_sensor_indices'] = nearest_sensors
    return census_tract_df
    
def census_tract_values(census_tract_df, la_county_sensors_df):
    pm_cols = []
    for index, row in census_tract_df.iterrows():
        closest_vals = []

        for sensor_index in row['nearest_sensor_indices']:
            sensor = la_county_sensors_df.loc[sensor_index]
            closest_vals.append(sensor['pm'])

        pm_cols.append(closest_vals)
    census_tract_df['pm_vals'] = pm_cols
    
    # Calculate weighted average
    census_pm = []

    for index, row in census_tract_df.iterrows():
        distances = []
        values = []
        polygon = row['shape']
        for sensor_index in row['nearest_sensor_indices']:
            sensor = la_county_sensors_df.loc[sensor_index]
            distances.append(1/polygon.centroid.distance(sensor.point))
            values.append(sensor['pm'])

        weighted_avg = np.average(values, weights=distances)

        census_pm.append(weighted_avg)

    census_tract_df['avg_pm'] = census_pm
    return census_tract_df

## PurpleAir Sensors

In [4]:
# Get PurpleAir sensor data
sensors_df = sensor_data()

In [5]:
# Get PurpleAir sensors in LA County
la_county_sensors_df = los_angeles_county_sensors(sensors_df)
la_county_sensors_df['point'] = la_county_sensors_df.apply(lambda x: Point(x['Lon'], x['Lat']), axis=1)

## LA County Census Tracts

In [6]:
census_tract_df = la_county_census_tracts()

## Find Nearest Sensors

In [7]:
census_tract_df = find_nearest_sensors(la_county_sensors_df, census_tract_df, 3)

## Compute Census Tract Values

In [8]:
census_tract_df = census_tract_values(census_tract_df, la_county_sensors_df)

## Save to geojson

In [9]:
geojson = {'type':'FeatureCollection', 'features':[]}
for tract, row in census_tract_df.iterrows():
    feature = {'type': 'Feature',
              'properties': {},
              'geometry': {'type': 'MultiPolygon',
                          'coordinates': []}}
    feature['geometry']['coordinates'] = row['coordinates']
    feature['properties']['tract'] = '0' + str(tract)
    feature['properties']['pm_25_weighted_avg'] = row['avg_pm']
    geojson['features'].append(feature)

In [10]:
output_filename = 'data.geojson'
with open(output_filename, 'w') as output_file:
    json.dump(geojson, output_file, indent=2) 